In [219]:
import pandas as pd 
import numpy as np
%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt
matplotlib.rcParams['figure.dpi'] = 200
plt.style.use('seaborn')
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [220]:
age = np.arange(1,120)
df = {'genomic instability':5+0.01*np.power(age,2),
     'loss of proteostasis':5+0.05*np.power(age,1.5),
     'stem cell exhaustion':5+0.05*np.power(age,1.5),
     'epigenetic alterations':5+0.05*np.power(age,1.5),
     'mitochondrial dysfunction':5+0.05*np.power(age,1.5),
     'telomere attrition':5+0.05*np.power(age,1.5),
     'cellular senescence':5+np.power(age,1.05)
     }

interaction_term = 0

df = pd.DataFrame(df)
df['interaction'] = df.sum(axis=1)*0.1
reset_df = df.copy()

total = df.iloc[-1,:].sum()
threshold = total/2

In [221]:
def plot_areas(intervention_age = 60, variable = 'genomic instability', therapeutic_factor = 0.0, colormap='viridis', multiple = False, reset = False): 
    
    global df
    temp_df = reset_df.copy()
    
    if reset: df = reset_df.copy()
        
    temp_y = temp_df[variable].values 
    temp_y[intervention_age:] *= (1.0 - therapeutic_factor)
    
    global threshold
    
    if not multiple:
        temp_df[variable] = temp_y
        ax = temp_df.plot.area(colormap=colormap,figsize=(10,5),linewidth=0)
        death_age = next(x[0] for x in enumerate(temp_df.sum(axis=1)) if x[1] > threshold)
    else:
        df[variable] = temp_y
        ax = df.plot.area(colormap=colormap,figsize=(10,5),linewidth=0)
        death_age = next(x[0] for x in enumerate(df.sum(axis=1)) if x[1] > threshold)
        
    ax.collections[-1].set_alpha(0.5)
    ax.collections[-1].set_hatch('\\')
    ax.collections[-1].set_color('gray')
    
    leg = plt.legend()
    leg.legendHandles[-1].set_alpha(0.5)
    
    

    plt.axhline(y=threshold, color='r', linestyle='--')
    plt.scatter(death_age,threshold,marker='.',c='red',s=80)
    plt.text(death_age,threshold*1.05,str(death_age),fontsize=10,color='red')
    plt.text(2, threshold*1.05, 'YOU DIE AT:', fontsize = 10, color='red')
    plt.xlim(0,len(age))
    plt.ylim(0,total)
    plt.xlabel('age')
    plt.ylabel('cellular damage')
    plt.title('Accumulated Cellular Damage')

In [222]:
_ = interact(plot_areas,
             colormap = widgets.Dropdown(
                 options = ['magma','viridis','plasma','inferno','cividis']
                 ),
             intervention_age = widgets.IntSlider(
                 value = 60,
                 min = 0,
                 max = 120,
                 step = 10
                ),
             variable = widgets.Dropdown(
                 options = df.columns.tolist(),value='epigenetic alterations'
                 ),
             therapeutic_factor = widgets.FloatSlider(
                value = 0.0, 
                min = 0.0,
                max = 1.0,
                step = 0.1,
                readout_format='.1f')
            )

interactive(children=(IntSlider(value=60, description='intervention_age', max=120, step=10), Dropdown(descript…